In [1]:
using Printf
Q = [5 -1; -1 2];
b = [1;1];

tol = 1e-10;
MaxIter = 1e4;

In [2]:
# define function
function F(x)
   val = 0.5*x'*Q*x - b'*x + 1
   return val
end

# define gradient
function DF(x)
   grad = Q*x-b;
   return grad
end

DF (generic function with 1 method)

In [3]:
function SteepestDescent(x0,alpha)

   # setup for steepest descent
   x = x0;
   successflag = false;

   # perform steepest descent iterations
   for iter = 1:MaxIter
       Fval = F(x);
       Fgrad = DF(x);
       if sqrt(Fgrad'*Fgrad) < tol
          @printf("Converged after %d iterations, function value %f\n", iter, Fval)
          successflag = true;
          break;
       end
       # perform steepest descent step
       x = x - alpha*Fgrad;

   end
   if successflag == false
       @printf("Failed to converge after %d iterations, function value %f\n", MaxIter, F(x))
   end
   return x;
end

SteepestDescent (generic function with 1 method)

In [4]:
# Q4A
x0 = [2;2];
q4a_alpha = [0.01, 0.05, 0.1,0.5];

for alpha in q4a_alpha
    print("alpha: ", alpha, "\n")
    x = SteepestDescent(x0,alpha);
end

alpha: 0.01
Converged after 1411 iterations, function value 0.500000
alpha: 0.05
Converged after 273 iterations, function value 0.500000
alpha: 0.1
Converged after 131 iterations, function value 0.500000
alpha: 0.5
Failed to converge after 10000 iterations, function value NaN


In [5]:
# 4B
using LinearAlgebra
Q_eig = eigvals(Q);
Q_eig

2-element Vector{Float64}:
 1.6972243622680054
 5.302775637731995

In [14]:
function SteepestDescentOptStep(x0)
    # setup for steepest descent
    x = x0
    successflag = false
    
    # perform steepest descent iterations
    for iter = 1:MaxIter
        Fval = F(x)
        Fgrad = DF(x)
        
        if sqrt(transpose(Fgrad) * Fgrad) < tol
            @printf("Converged after %d iterations, function value %f\n", iter, Fval)
            successflag = true
            break
        end
        
        # Finding Optimal Step
        opt_step = (transpose(Fgrad) * Fgrad) / (transpose(Fgrad) * Q * Fgrad)
        
        # perform steepest descent step
        x = x - opt_step * Fgrad
    end
    
    if successflag == false
        @printf("Failed to converge after %d iterations, function value %f\n", MaxIter, F(x))
    end
    
    return x
end

SteepestDescentOptStep (generic function with 1 method)

In [15]:
# 4C
x0 = [2; 2]
x = SteepestDescentOptStep(x0)

Converged after 30 iterations, function value 0.500000


2-element Vector{Float64}:
 0.3333333333373737
 0.6666666666941405

In [8]:
function SteepestDescentArmijo(x0, c1)

   # parameters for Armijo's rule
   alpha0 = 5;    # initial value of alpha, to try in backtracking
   eta = 0.5;       # factor with which to scale alpha, each time you backtrack
   MaxBacktrack = 20;  # maximum number of backtracking steps

   # setup for steepest descent
   x = x0;
   successflag = false;   

   # perform steepest descent iterations
   for iter = 1:MaxIter

      alpha = alpha0;
      Fval = F(x);
      Fgrad = DF(x);

      # check if norm of gradient is small enough
      if sqrt(Fgrad'*Fgrad) < tol
         @printf("Converged after %d iterations, function value %f\n", iter, Fval)
         successflag = true;
         break;
      end

      # perform line search
      for k = 1:MaxBacktrack
         x_try = x - alpha*Fgrad;
         Fval_try = F(x_try);
         if (Fval_try > Fval - c1* alpha * Fgrad' * Fgrad)
            alpha = alpha * eta;
         else
            Fval = Fval_try;
            x = x_try;
            break;
         end
      end

      # print how we're doing, every 10000 iterations 
        #(I modified it so that I could take a screenshot of the code for when it doesn't converge)
      if (iter%10000==0)
         @printf("iter: %d: alpha: %f, %f, %f, %f\n", iter, alpha, x[1], x[2], Fval)
      end
   end

   if successflag == false
       @printf("Failed to converge after %d iterations, function value %f\n", MaxIter, F(x))
   end

   return x;
end

SteepestDescentArmijo (generic function with 1 method)

In [9]:
#4D
x0 = [2;2];
Q4d_c_list = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.2]

for c1 in Q4d_c_list
    print("c1: ", c1, "\n")
    y = SteepestDescentArmijo(x0, c1);
end

c1: 1.0e-5
Converged after 69 iterations, function value 0.500000
c1: 0.0001
Converged after 69 iterations, function value 0.500000
c1: 0.001
Converged after 69 iterations, function value 0.500000
c1: 0.01
Converged after 69 iterations, function value 0.500000
c1: 0.1
Converged after 69 iterations, function value 0.500000
c1: 0.2
iter: 10000: alpha: 0.000005, 0.333333, 0.666667, 0.500000
Failed to converge after 10000 iterations, function value 0.500000


## Question 5 - Rosebrock Function

In [10]:
# Set up Parameters
tol = 1e-10
MaxIter = 1e5

# define Rosenbrock Function
function rb_F(x)
    a = x[1]
    b = x[2]
    val = 100 * (b - a^2)^2 + (1 - a)^2
    return val
end

# define gradient of Rosenbrock
function rb_DF(x)
    a = x[1]
    b = x[2]
    
    df_da = -400 * a * (b - a^2) - 2 * (1 - a)
    df_db = 200 * (b - a^2)
    
    grad = [df_da, df_db]
    return grad
end

# Steepest Descent Algorithm

function RB_SteepestDescent(x0,alpha)
    
    x = x0
    successflag = false  # Define successflag

    for iter = 1:MaxIter
        Fval = rb_F(x)
        Fgrad = rb_DF(x)
        if sqrt(transpose(Fgrad) * Fgrad) < tol
            @printf("Converged after %d iterations, function value %f\n", iter, Fval)
            successflag = true
            break
        end
        # perform steepest descent step
        x = x - alpha * Fgrad
    end

    if !successflag
        @printf("Failed to converge after %d iterations, function value %f\n", MaxIter, rb_F(x))
    end
    
    return x;
end 

RB_SteepestDescent (generic function with 1 method)

In [11]:
x0 = [2.0; 2.0]
Q5_alpha = [0.00001, 0.0005, 0.0009, 0.001, 0.0011,0.002, 0.01, 0.1]

for alpha in Q5_alpha
    print("Alpha = ", alpha, "\n")
    x = RB_SteepestDescent(x0,alpha);
end

Alpha = 1.0e-5
Failed to converge after 100000 iterations, function value 0.140645
Alpha = 0.0005
Failed to converge after 100000 iterations, function value 0.000000
Alpha = 0.0009
Converged after 62405 iterations, function value 0.000000
Alpha = 0.001
Converged after 55332 iterations, function value 0.000000
Alpha = 0.0011
Converged after 48713 iterations, function value 0.000000
Alpha = 0.002
Failed to converge after 100000 iterations, function value NaN
Alpha = 0.01
Failed to converge after 100000 iterations, function value NaN
Alpha = 0.1
Failed to converge after 100000 iterations, function value NaN


In [12]:
# Set up Parameters
tol = 1e-10
MaxIter = 1e5

# Steepest Descent Algorithm with Armijo's rule for step size
function RB_SteepestDescentArmijo(x0, c1)
    x = x0
    successflag = false  # Define successflag
    
    # parameters for Armijo's rule
    alpha0 = 1.0  # initial value of alpha, can be adjusted
    eta = 0.5     # factor with which to scale alpha, each time you backtrack
    MaxBacktrack = 20  # maximum number of backtracking steps

    for iter = 1:MaxIter
        Fval = rb_F(x)
        Fgrad = rb_DF(x)
        
        # check if norm of gradient is small enough
        if sqrt(transpose(Fgrad) * Fgrad) < tol
            @printf("Converged after %d iterations, function value %f\n", iter, Fval)
            successflag = true
            break
        end

        alpha = alpha0
        # perform backtracking line search (Armijo rule)
        for k = 1:MaxBacktrack
            x_try = x - alpha * Fgrad
            Fval_try = rb_F(x_try)
            if Fval_try <= Fval - c1 * alpha * transpose(Fgrad) * Fgrad
                x = x_try
                break
            else
                alpha = alpha * eta
            end
        end

        # print how we're doing, every 10 iterations
        # Note: I turned this section off so that I could actually see the results
        #if iter % 10 == 0
        #    @printf("iter: %d: alpha: %f, %f, %f, %f\n", iter, alpha, x[1], x[2], Fval)
        #end
    end

    if !successflag
        @printf("Failed to converge after %d iterations, function value %f\n", MaxIter, rb_F(x))
    end
    
    return x
end


RB_SteepestDescentArmijo (generic function with 1 method)

In [13]:
# 5B
x0 = [2.0;2.0];
Q5b_c_list = [1e-5, 1e-3, 1e-1]

for c1 in Q5b_c_list
    print("c1: ", c1, "\n")
    y = RB_SteepestDescentArmijo(x0, c1);
end

c1: 1.0e-5
Converged after 29013 iterations, function value 0.000000
c1: 0.001
Converged after 28940 iterations, function value 0.000000
c1: 0.1
Converged after 25863 iterations, function value 0.000000
